In [ ]:
# Importing Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!wget --header="Host: doc-08-74-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://drive.google.com/" --header="Cookie: AUTH_f75odganf668fkra51jtfs8k051cseiq_nonce=sicukf7b533eu" --header="Connection: keep-alive" "https://doc-08-74-docs.googleusercontent.com/docs/securesc/bkov7h20sfrps3mlq50da5p22h75grj3/0bkqpjpe6fl14skv105tf6i8koeuf9u6/1595602275000/06629147635963609455/06629147635963609455/118toouH8ifByKJHmmowyjOjGyir-YAvi?e=download&h=16381748609525104559&authuser=0&nonce=sicukf7b533eu&user=06629147635963609455&hash=1p8s4rpp0oo2rl44d67dr0r7n3s5ctmd" -c -O 'HumanActivityRecognition.zip'

--2020-08-04 09:53:45--  https://doc-08-74-docs.googleusercontent.com/docs/securesc/bkov7h20sfrps3mlq50da5p22h75grj3/0bkqpjpe6fl14skv105tf6i8koeuf9u6/1595602275000/06629147635963609455/06629147635963609455/118toouH8ifByKJHmmowyjOjGyir-YAvi?e=download&h=16381748609525104559&authuser=0&nonce=sicukf7b533eu&user=06629147635963609455&hash=1p8s4rpp0oo2rl44d67dr0r7n3s5ctmd
Resolving doc-08-74-docs.googleusercontent.com (doc-08-74-docs.googleusercontent.com)... 74.125.31.132, 2607:f8b0:400c:c02::84
Connecting to doc-08-74-docs.googleusercontent.com (doc-08-74-docs.googleusercontent.com)|74.125.31.132|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2020-08-04 09:53:45 ERROR 403: Forbidden.



In [ ]:
!unzip -qn HumanActivityRecognition.zip

[HumanActivityRecognition.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of HumanActivityRecognition.zip or
        HumanActivityRecognition.zip.zip, and cannot find HumanActivityRecognition.zip.ZIP, period.


In [ ]:
!ls HAR

ls: cannot access 'HAR': No such file or directory


In [3]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [5]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [6]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/My Drive/HAR/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [7]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/drive/My Drive/HAR/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [9]:
# Importing tensorflow
%tensorflow_version 2.x
import numpy as np
np.random.seed(42)
from keras import backend as K
import tensorflow as tf
print(tf.__git_version__)

#tf.compat.v1.enable_eager_execution()


#tf.set_random_seed(42)


v2.3.0-0-gb36436b087


In [10]:
# Importing libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM ,BatchNormalization
from tensorflow.keras.layers import Dense, Dropout

In [11]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [12]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [13]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [14]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [15]:
tf.__version__

'2.3.0'

In [16]:
timesteps

128

In [17]:
import tensorflow as tf

tf.__version__

'2.3.0'

In [18]:
tf.keras.backend.clear_session()

In [19]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(32, input_shape=(128, 9)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5376      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=5)

Epoch 1/5
460/460 [==============================] - 21s 47ms/step - loss: 1.4092 - accuracy: 0.3716 - val_loss: 1.3050 - val_accuracy: 0.4737
Epoch 2/5
460/460 [==============================] - 21s 47ms/step - loss: 1.0931 - accuracy: 0.5400 - val_loss: 0.9484 - val_accuracy: 0.6010
Epoch 3/5
460/460 [==============================] - 21s 46ms/step - loss: 0.8189 - accuracy: 0.6285 - val_loss: 0.8053 - val_accuracy: 0.6366
Epoch 4/5
460/460 [==============================] - 21s 47ms/step - loss: 0.7292 - accuracy: 0.6616 - val_loss: 0.8020 - val_accuracy: 0.6176
Epoch 5/5
460/460 [==============================] - 21s 46ms/step - loss: 0.6643 - accuracy: 0.6953 - val_loss: 0.7718 - val_accuracy: 0.6854


In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

In [ ]:
score = model.evaluate(X_test, Y_test)

In [ ]:
score

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

# Assignment


# Model 1

In [ ]:
model = Sequential()
model.add(LSTM(150, input_shape=(128, 9),return_sequences=True,))
model.add(BatchNormalization())
model.add(Dropout(0.65))
model.add(LSTM(75))
model.add(Dropout(0.65))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128, 150)          96000     
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 150)          600       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 150)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 75)                67800     
_________________________________________________________________
dropout_2 (Dropout)          (None, 75)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 456       
Total params: 164,856
Trainable params: 164,556
Non-trainable params: 300
______________________________

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=10)

Epoch 1/10
460/460 [==============================] - 96s 209ms/step - loss: 0.8286 - accuracy: 0.6766 - val_loss: 1.2306 - val_accuracy: 0.6267
Epoch 2/10
460/460 [==============================] - 96s 208ms/step - loss: 0.3679 - accuracy: 0.8792 - val_loss: 0.3849 - val_accuracy: 0.8836
Epoch 3/10
460/460 [==============================] - 96s 209ms/step - loss: 0.2500 - accuracy: 0.9168 - val_loss: 0.6072 - val_accuracy: 0.8415
Epoch 4/10
460/460 [==============================] - 96s 208ms/step - loss: 0.2057 - accuracy: 0.9328 - val_loss: 0.2540 - val_accuracy: 0.9111
Epoch 5/10
460/460 [==============================] - 96s 210ms/step - loss: 0.1862 - accuracy: 0.9350 - val_loss: 0.4412 - val_accuracy: 0.8826
Epoch 6/10
460/460 [==============================] - 95s 207ms/step - loss: 0.1810 - accuracy: 0.9370 - val_loss: 0.5522 - val_accuracy: 0.8656
Epoch 7/10
460/460 [==============================] - 95s 208ms/step - loss: 0.1813 - accuracy: 0.9359 - val_loss: 0.3442 - val_ac

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 537        0  ...                   0                 0
SITTING                 22      412  ...                   0                 3
STANDING                 0       75  ...                   0                 2
WALKING                  0        0  ...                  20                37
WALKING_DOWNSTAIRS       0        0  ...                 416                 4
WALKING_UPSTAIRS         0        0  ...                  13               458

[6 rows x 6 columns]


In [ ]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 6s 67ms/step - loss: 0.3367 - accuracy: 0.9220


In [ ]:
score

[0.33674168586730957, 0.9219545125961304]

# Model 2 

In [ ]:
model = Sequential()
model.add(LSTM(150, input_shape=(128, 9),return_sequences=True,))
model.add(BatchNormalization())
model.add(Dropout(0.65))
model.add(LSTM(75))
model.add(BatchNormalization())
model.add(Dropout(0.65))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128, 150)          96000     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 150)          600       
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 150)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 75)                67800     
_________________________________________________________________
batch_normalization_2 (Batch (None, 75)                300       
_________________________________________________________________
dropout_4 (Dropout)          (None, 75)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=10)

Epoch 1/10
460/460 [==============================] - 98s 213ms/step - loss: 0.9802 - accuracy: 0.6243 - val_loss: 1.2787 - val_accuracy: 0.5952
Epoch 2/10
460/460 [==============================] - 97s 210ms/step - loss: 0.7420 - accuracy: 0.6708 - val_loss: 0.7906 - val_accuracy: 0.6166
Epoch 3/10
460/460 [==============================] - 96s 209ms/step - loss: 0.6746 - accuracy: 0.7104 - val_loss: 0.7510 - val_accuracy: 0.6868
Epoch 4/10
460/460 [==============================] - 96s 210ms/step - loss: 0.5468 - accuracy: 0.7918 - val_loss: 0.5247 - val_accuracy: 0.8073
Epoch 5/10
460/460 [==============================] - 98s 212ms/step - loss: 0.3705 - accuracy: 0.8853 - val_loss: 0.3494 - val_accuracy: 0.8809
Epoch 6/10
460/460 [==============================] - 102s 222ms/step - loss: 0.2974 - accuracy: 0.9061 - val_loss: 0.4702 - val_accuracy: 0.8778
Epoch 7/10
460/460 [==============================] - 97s 211ms/step - loss: 0.2587 - accuracy: 0.9166 - val_loss: 0.7198 - val_a

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 537        0  ...                   0                 0
SITTING                  3      288  ...                   0                 1
STANDING                 0       25  ...                   0                 0
WALKING                  0        0  ...                  13                 2
WALKING_DOWNSTAIRS       0        0  ...                 418                 1
WALKING_UPSTAIRS         0        0  ...                   2               440

[6 rows x 6 columns]


In [ ]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 6s 67ms/step - loss: 0.2669 - accuracy: 0.9063


In [ ]:
score

[0.2668585777282715, 0.9063454270362854]

# Model 3 

In [ ]:
model = Sequential()
model.add(LSTM(150, input_shape=(128, 9),return_sequences=True,))
model.add(BatchNormalization())
model.add(Dropout(0.7))
model.add(LSTM(75))
model.add(Dropout(0.7))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128, 150)          96000     
_________________________________________________________________
batch_normalization_3 (Batch (None, 128, 150)          600       
_________________________________________________________________
dropout_5 (Dropout)          (None, 128, 150)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 75)                67800     
_________________________________________________________________
dropout_6 (Dropout)          (None, 75)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 456       
Total params: 164,856
Trainable params: 164,556
Non-trainable params: 300
______________________________

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=10)

Epoch 1/10
460/460 [==============================] - 97s 212ms/step - loss: 0.2965 - accuracy: 0.6253 - val_loss: 0.2501 - val_accuracy: 0.6128
Epoch 2/10
460/460 [==============================] - 101s 219ms/step - loss: 0.1499 - accuracy: 0.8750 - val_loss: 0.1379 - val_accuracy: 0.8392
Epoch 3/10
460/460 [==============================] - 97s 210ms/step - loss: 0.0987 - accuracy: 0.9200 - val_loss: 0.1974 - val_accuracy: 0.8310
Epoch 4/10
460/460 [==============================] - 97s 212ms/step - loss: 0.0832 - accuracy: 0.9282 - val_loss: 0.0775 - val_accuracy: 0.9226
Epoch 5/10
460/460 [==============================] - 97s 211ms/step - loss: 0.0726 - accuracy: 0.9376 - val_loss: 0.1895 - val_accuracy: 0.8083
Epoch 6/10
460/460 [==============================] - 97s 212ms/step - loss: 0.0751 - accuracy: 0.9340 - val_loss: 0.1160 - val_accuracy: 0.9199
Epoch 7/10
460/460 [==============================] - 98s 212ms/step - loss: 0.0714 - accuracy: 0.9338 - val_loss: 0.1303 - val_a

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 526       11  ...                   0                 0
SITTING                  1      364  ...                   0                 3
STANDING                 0       47  ...                   0                 0
WALKING                  0        0  ...                  23                 4
WALKING_DOWNSTAIRS       0        0  ...                 411                 8
WALKING_UPSTAIRS         0        0  ...                  23               448

[6 rows x 6 columns]


In [ ]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 6s 68ms/step - loss: 0.1201 - accuracy: 0.9172


In [ ]:
score

[0.12010446935892105, 0.917203962802887]

# Model 4 with L1L2 regularizers

In [20]:
from tensorflow.keras.regularizers import L1L2
reg = L1L2(0.01, 0.01)

In [ ]:
model = Sequential()
model.add(LSTM(120, input_shape=(128, 9),return_sequences=True,bias_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(0.75))
model.add(LSTM(60))
model.add(Dropout(0.75))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 128, 120)          62400     
_________________________________________________________________
batch_normalization_8 (Batch (None, 128, 120)          480       
_________________________________________________________________
dropout_15 (Dropout)         (None, 128, 120)          0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 60)                43440     
_________________________________________________________________
dropout_16 (Dropout)         (None, 60)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 366       
Total params: 106,686
Trainable params: 106,446
Non-trainable params: 240
______________________________

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=20)

Epoch 1/20
460/460 [==============================] - 80s 174ms/step - loss: 1.9809 - accuracy: 0.5736 - val_loss: 1.2331 - val_accuracy: 0.6485
Epoch 2/20
460/460 [==============================] - 80s 175ms/step - loss: 0.7439 - accuracy: 0.7311 - val_loss: 0.2805 - val_accuracy: 0.8029
Epoch 3/20
460/460 [==============================] - 81s 176ms/step - loss: 0.1932 - accuracy: 0.8415 - val_loss: 0.1223 - val_accuracy: 0.8697
Epoch 4/20
460/460 [==============================] - 81s 176ms/step - loss: 0.1356 - accuracy: 0.9008 - val_loss: 0.1561 - val_accuracy: 0.8208
Epoch 5/20
460/460 [==============================] - 81s 176ms/step - loss: 0.1107 - accuracy: 0.9181 - val_loss: 0.1177 - val_accuracy: 0.8955
Epoch 6/20
460/460 [==============================] - 83s 180ms/step - loss: 0.0951 - accuracy: 0.9320 - val_loss: 0.1775 - val_accuracy: 0.8453
Epoch 7/20
460/460 [==============================] - 82s 178ms/step - loss: 0.0892 - accuracy: 0.9319 - val_loss: 0.0940 - val_ac

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 537        0  ...                   0                 0
SITTING                  2      309  ...                   0                 0
STANDING                 0       45  ...                   0                 0
WALKING                  0        0  ...                   7                 7
WALKING_DOWNSTAIRS       0        0  ...                 415                 4
WALKING_UPSTAIRS         0        0  ...                  24               442

[6 rows x 6 columns]


In [ ]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 5s 53ms/step - loss: 0.1141 - accuracy: 0.9067


In [ ]:
score

[0.1141052171587944, 0.9066847562789917]

# Model 5

In [28]:
model = Sequential()
model.add(LSTM(100, input_shape=(timesteps, input_dim), kernel_initializer='glorot_normal' , return_sequences=True, bias_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(0.70))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.70))
model.add(LSTM(25))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 128, 100)          44000     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 100)          400       
_________________________________________________________________
dropout_5 (Dropout)          (None, 128, 100)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 128, 50)           30200     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128, 50)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 25)                7600      
_________________________________________________________________
dense_2 (Dense)              (None, 6)

In [36]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [38]:
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=15)

Epoch 1/15
460/460 [==============================] - 82s 178ms/step - loss: 0.0358 - accuracy: 0.9563 - val_loss: 0.1207 - val_accuracy: 0.9040
Epoch 2/15
460/460 [==============================] - 82s 179ms/step - loss: 0.0329 - accuracy: 0.9589 - val_loss: 0.0983 - val_accuracy: 0.9111
Epoch 3/15
460/460 [==============================] - 82s 179ms/step - loss: 0.0358 - accuracy: 0.9542 - val_loss: 0.1003 - val_accuracy: 0.9226
Epoch 4/15
460/460 [==============================] - 83s 181ms/step - loss: 0.0351 - accuracy: 0.9559 - val_loss: 0.1429 - val_accuracy: 0.8928
Epoch 5/15
460/460 [==============================] - 84s 182ms/step - loss: 0.0364 - accuracy: 0.9582 - val_loss: 0.0919 - val_accuracy: 0.9172
Epoch 6/15
460/460 [==============================] - 83s 181ms/step - loss: 0.0341 - accuracy: 0.9585 - val_loss: 0.0978 - val_accuracy: 0.9155
Epoch 7/15
460/460 [==============================] - 83s 180ms/step - loss: 0.0363 - accuracy: 0.9563 - val_loss: 0.0897 - val_ac

In [39]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 537        0  ...                   0                 0
SITTING                 21      421  ...                   0                 5
STANDING                 0       97  ...                   0                 0
WALKING                  0        0  ...                   3                 1
WALKING_DOWNSTAIRS       0        0  ...                 413                 7
WALKING_UPSTAIRS         0        0  ...                   8               459

[6 rows x 6 columns]


In [40]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 5s 51ms/step - loss: 0.0756 - accuracy: 0.9352


In [41]:
score

[0.07562656700611115, 0.935188353061676]

# Summary

In [43]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Model", "Test loss", "Test Accuracy"]
x.add_row(["1- 2 LSTM ,2 Dropout ,1BN , 10 epochs,categorical_crossentropy",0.33674168586730957, 0.9219545125961304])
x.add_row(["2- 2 LSTM ,2 Dropout ,2BN , 10 epochs,categorical_crossentropy",0.2668585777282715, 0.9063454270362854])
x.add_row(["3- 2 LSTM ,2 Dropout ,1BN , 10 epochs,binary_crossentropy",0.12010446935892105, 0.917203962802887])
x.add_row(["4- 2 LSTM ,2 Dropout ,1BN and L1L2 regularizer 20epochs, binary_crossentropy",0.1141052171587944, 0.9066847562789917])
x.add_row(["5- 3 LSTM ,2 Dropout ,1BN and L1L2 regularizer 15epochs, binary_crossentropy",0.07562656700611115, 0.935188353061676])
print(x)

+------------------------------------------------------------------------------+---------------------+--------------------+
|                                    Model                                     |      Test loss      |   Test Accuracy    |
+------------------------------------------------------------------------------+---------------------+--------------------+
|        1- 2 LSTM ,2 Dropout ,1BN , 10 epochs,categorical_crossentropy        | 0.33674168586730957 | 0.9219545125961304 |
|        2- 2 LSTM ,2 Dropout ,2BN , 10 epochs,categorical_crossentropy        |  0.2668585777282715 | 0.9063454270362854 |
|          3- 2 LSTM ,2 Dropout ,1BN , 10 epochs,binary_crossentropy           | 0.12010446935892105 | 0.917203962802887  |
| 4- 2 LSTM ,2 Dropout ,1BN and L1L2 regularizer 20epochs, binary_crossentropy |  0.1141052171587944 | 0.9066847562789917 |
| 5- 3 LSTM ,2 Dropout ,1BN and L1L2 regularizer 15epochs, binary_crossentropy | 0.07562656700611115 | 0.935188353061676  |
+-------